In [39]:
import permutationTest as pt 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time
import pickle

In [35]:
sampleShape = [10,50,100,150,200,250,300,400,500]
n_samples = 1
bins =  500
mean =0.2
std = 1

data_variables = (sampleShape, n_samples, bins, mean, std)

In [36]:
data_generator = lambda mean, std, size ,n_samples : [np.random.normal(mean, std, size) for _ in range(n_samples)]


In [37]:
def runExperiment(method_list, names_list, data_generator, data_variables):
    sampleShape, n_samples, bins, mean, std = data_variables
    
    method_dict = dict()
    
    for method, name in zip(method_list, names_list):
        
        method_dict[name] = dict()
        method_dict[name]["p_values"] = list()
        method_dict[name]["time"] = list()
        
        for size in sampleShape:
            
            np.random.seed(42)
            A = np.asarray(data_generator(mean, std, size ,n_samples))
            B = np.asarray(data_generator(mean, std, size ,n_samples))
        
            start = time.time()
            method_dict[name]["p_values"].append(method(A,B,bins)[0])
            end = time.time()
            method_dict[name]["time"].append(end - start)
            
    return method_dict
    

In [40]:
method_list = [pt.GreenFloatCuda]
names_list = ["RTX20170"]
RTX20170_dict = runExperiment(method_list, names_list, data_generator, data_variables)

In [42]:
pickle.dump(RTX20170_dict,open("./data/experiment2/RTX20170_dict.pkl", "wb"))

In [19]:
data(mean,std,10,n_samples)

[array([ 1.87336657, -0.06767134, -0.15160338,  1.15279127,  0.14945245,
         0.1325056 , -0.821861  , -0.41059903,  1.04042678,  0.66131615])]

In [13]:
data = lambda n,n_samples : [np.random.randint(0,n,n,dtype=np.int32) for _ in range(n_samples)]

In [14]:
data(10,1)

[array([7, 7, 5, 4, 2, 6, 8, 8, 1, 2], dtype=int32)]